## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-27-10-52-00 +0000,wsj,Sweden’s Debt Office Expects Widening Budget D...,https://www.wsj.com/economy/central-banking/sw...
1,2025-11-27-10-50-43 +0000,nyt,Live Updates: At Least 55 Dead in Hong Kong Ap...,https://www.nytimes.com/live/2025/11/27/world/...
2,2025-11-27-10-48-14 +0000,nyt,"Pope Leo Lands in Turkey, Planning to Meet Erd...",https://www.nytimes.com/2025/11/27/world/middl...
3,2025-11-27-10-48-00 +0000,wsj,Eurozone Business Confidence Holds Steady,https://www.wsj.com/economy/eurozone-business-...
4,2025-11-27-10-47-22 +0000,nypost,Louisiana death row inmate freed after nearly ...,https://nypost.com/2025/11/27/us-news/louisian...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,37
328,national,26
17,fire,19
14,hong,18
15,kong,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
313,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,117
117,2025-11-27-00-27-54 +0000,nypost,"Afghan national Rahmanullah Lakanwal, who came...",https://nypost.com/2025/11/26/us-news/afghan-n...,116
159,2025-11-26-21-59-46 +0000,nypost,Trump requests 500 more National Guard troops ...,https://nypost.com/2025/11/26/us-news/trump-re...,111
75,2025-11-27-05-08-01 +0000,startribune,Trump targets Somali community in Minnesota an...,https://www.startribune.com/trump-says-lax-mig...,110
84,2025-11-27-04-06-00 +0000,wsj,Trump Calls D.C. National Guard Shooting ‘Act ...,https://www.wsj.com/us-news/national-guard-sho...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
313,117,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
54,83,2025-11-27-07-59-00 +0000,wsj,Three Arrested as Hong Kong Fire Death Toll Ri...,https://www.wsj.com/world/asia/hong-kong-fire-...
117,64,2025-11-27-00-27-54 +0000,nypost,"Afghan national Rahmanullah Lakanwal, who came...",https://nypost.com/2025/11/26/us-news/afghan-n...
94,57,2025-11-27-02-53-50 +0000,nypost,"Hunter, Joe Biden pictured together for first ...",https://nypost.com/2025/11/26/us-news/hunter-j...
267,49,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...
186,42,2025-11-26-20-33-00 +0000,wsj,The job market continues to sputter even as co...,https://www.wsj.com/economy/central-banking/fe...
33,39,2025-11-27-10-00-00 +0000,wsj,"Centrist dealmaker Sen. Jeanne Shaheen (D., N....",https://www.wsj.com/politics/policy/jeanne-sha...
244,36,2025-11-26-17-31-50 +0000,nyt,Trump Defends Witkoff’s Tactics in Russia-Ukra...,https://www.nytimes.com/2025/11/26/us/politics...
61,35,2025-11-27-07-10-43 +0000,nypost,Virginia football coach Travis Turner wanted f...,https://nypost.com/2025/11/27/us-news/virginia...
203,35,2025-11-26-19-26-47 +0000,nypost,Pope Leo XIV embarking on first foreign trip t...,https://nypost.com/2025/11/26/world-news/pope-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
